# Share Analysis

### Extract data from db and convert it in a seaborn compatible data source

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from voices.website import models

In [ ]:
shares = models.Share.objects.order_by("timestamp").all()

df = pd.DataFrame({
    "longitude": pd.Series(dtype='float'),
    "latitude": pd.Series(dtype='float'),
    "datetime": pd.Series(dtype="datetime64[ns, UTC]"),
    "timestamp": pd.Series(dtype='int'),
    "message": pd.Series(dtype='str'),
})

for share in models.Share.objects.order_by("timestamp").all():
    x, y = share.mercator_coordinates
    ts = share.timestamp.timestamp() // 1000 
    df.loc[len(df)] = [
        x, y, share.timestamp, ts, share.message
    ]
    
df.head()

### Bokeh 

In [ ]:
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource
from bokeh.palettes import YlOrRd
from bokeh.plotting import curdoc, figure, show
from bokeh.transform import linear_cmap

import xyzservices.providers as xyz

output_notebook()
curdoc().theme = 'dark_minimal'

In [ ]:
def mercator_longitude(longitude) -> float:
    return longitude * (6378137 * np.pi / 180.0)

def mercator_latitude(latitude) -> float:
    return np.log(np.tan((90 + latitude) * np.pi / 360.0)) * 6378137

def mercator_projection(coord: tuple[float, float]):
    x = mercator_longitude(coord[1])
    y = mercator_latitude(coord[0])
    return x, y

In [ ]:
p = figure(
    match_aspect=True, 
    aspect_ratio=2, 
    sizing_mode="stretch_width",
    x_axis_type="mercator", 
    y_axis_type="mercator",
    x_range=(mercator_longitude(10.88), mercator_longitude(10.91)), 
    y_range=(mercator_latitude(44.625), mercator_latitude(44.665)),
    tooltips=[("message", "@message")],
)
p.add_tile(xyz.Stamen.TonerBackground, retina=True)

In [ ]:
source = ColumnDataSource(df)

cmap = linear_cmap(
    "timestamp", 
    "Inferno256", 
    low=df.timestamp.min(),
    high=df.timestamp.max(),
)

p.circle(
    source=source, 
    x="longitude", 
    y="latitude", 
    color=cmap,
    line_color="gray",
    size=12,
)

show(p)